```
➜  s git:(master) pyspark
Python 3.9.6 (default, Jun 29 2021, 06:20:32) 
[Clang 12.0.0 (clang-1200.0.32.29)] on darwin
Type "help", "copyright", "credits" or "license" for more information.
21/09/22 04:23:06 WARN Utils: Your hostname, kangui-MacBookPro.local resolves to a loopback address: 127.0.0.1; using 192.168.0.14 instead (on interface en0)
21/09/22 04:23:06 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
WARNING: An illegal reflective access operation has occurred
WARNING: Illegal reflective access by org.apache.spark.unsafe.Platform (file:/usr/local/lib/python3.9/site-packages/pyspark/jars/spark-unsafe_2.12-3.1.2.jar) to constructor java.nio.DirectByteBuffer(long,int)
WARNING: Please consider reporting this to the maintainers of org.apache.spark.unsafe.Platform
WARNING: Use --illegal-access=warn to enable warnings of further illegal reflective access operations
WARNING: All illegal access operations will be denied in a future release
21/09/22 04:23:07 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 3.1.2
      /_/

Using Python version 3.9.6 (default, Jun 29 2021 06:20:32)
Spark context Web UI available at http://192.168.0.14:4040
Spark context available as 'sc' (master = local[*], app id = local-1632252188331).
SparkSession available as 'spark'.
>>> exit

```

# 문제 1: 오픈API를 통해 데이터를 5천 ~ 1만 건 정도 수집해서 파일로 저장한다 (초과해도 좋다).

- 열린데이터, 공공데이터 또는 (할 수 있다면) 트위터와 같은 SNS, 댓글 등 적합한 소스를 자신이 선택한다.

- 과정의 후반으로 가서, 이 데이터를 분석하는 과제가 주어지게 된다.

- 프로그램에서 몇 건을 읽었는지와 마지막 줄을 화면 출력한다.

- !dir 명령어로 파일이 존재하는지 출력한다. 1만 건 정도 저장되었으니 파일의 크기가 적당한지 가늠해보자

# 문제 2: 노트북에서 pyspark 실행

주피터 노트북을 실행한 후, 셀에서 아래 명령어를 실행하여 spark를 생성하고, 그 버전을 출력하세요.

[In] spark.version

[Out] '3.x.x'



그리고 다음과 같이 버전을 출력한다.

pyspark는 jdk가 설치되어 있어야 한다.

!java --version   #spark v2.x는 java 8이 적당.

!javac --version  #java 버전과 동일한지 확인한다.

!python --version  

!python3 --version # python 2, 3 멀티 버전이 설치된 경우에만 출력

붙여서 제출해라?

In [42]:
spark.version

'3.1.2'

In [45]:
!java -version #spark v2.x는 java 8이 적당.

!javac -version #java 버전과 동일한지 확인한다.

!python --version

!python3 --version # python 2, 3 멀티 버전이 설치된 경우에만 출력

openjdk version "1.8.0_222"
OpenJDK Runtime Environment (AdoptOpenJDK)(build 1.8.0_222-b10)
OpenJDK 64-Bit Server VM (AdoptOpenJDK)(build 25.222-b10, mixed mode)
javac 1.8.0_222
Python 2.7.15
Python 3.9.5


# 문제 3: 다음 파일을 읽어서 RDD를 생성하고, 5줄을 화면출력하세요.

결과가 깨져보인다면, 그 이유를 간단히 적으세요.

* 1) 경기도 의정부시 인구현황 (파일명: ```경기도 의정부시_인구현황_20200904```)

https://www.data.go.kr/data/15009613/fileData.do





* 2) 제주특별자치도 서귀포시 내 연도별 65세이상 인구수 및 고령화비율, 노령화지수 현황 (파일명: ```제주특별자치도 서귀포시_고령화비율및노령화지수현황_20200623```)

https://www.data.go.kr/data/15051545/fileData.do



In [51]:
df1=spark.read.csv(os.path.join("data", "경기도 의정부시_인구현황_20210910.csv"))
df1.first()

Row(_c0='�������', _c1='�α���(��)', _c2='�α���(��)', _c3='�α���(��)', _c4='������(��)', _c5='������(��)', _c6='������(��)', _c7='����', _c8='�����', _c9='������α�', _c10='���������', _c11='�����μ���', _c12='�μ���ȭ��ȣ', _c13='�����ͱ�������')

In [49]:
df2=spark.read.csv(os.path.join("data", "제주특별자치도 서귀포시_고령화비율및노령화지수현황_20210831.csv"))
df2.first()

Row(_c0='������', _c1='�������� �α���', _c2='65���̻� �α��� ', _c3='14������ �α���', _c4='���ȭ����', _c5='���ȭ����', _c6='�����ͱ�������')

In [28]:
import os

In [21]:
import requests
import urllib


# 경기도 의정부시_인구현황
def fetchUijeongbuPopulationInformation():
    # https://www.data.go.kr/iim/api/selectAPIAcountView.do 
    API_URL = "https://api.odcloud.kr/api/15009613/v1/uddi:a3c28072-99fc-4e37-9113-8e81cab51ab9"
    
    # Decoding
    API_KEY = "8OGTO7jd9Ih3JPZej0XBI0ITT2g6v1f1zxq7TibztJxeUYTyVjqSYAqePFeUP1eX7D5r2EKdlWU3w6Wfu9fKug=="
    
    # Parameters 설정
    # page=1&perPage=10 
    _d=dict()
    _d['page']= '1'
    _d['perPage']= '1'
    _d['serviceKey']= API_KEY
    _d['returnType']= 'JSON'
    
    # url encoding
    params=urllib.parse.urlencode(_d)
    
    # header
    headers = {
        "Authorization": API_KEY,
        "Accept": "application/json",
        "Content-Type": "application/json; charset=UTF-8"
    }
    
    # data fetching
    res=requests.get(API_URL + '?' + params, headers=headers)
    
    return res.json()

In [34]:
import requests
import urllib


# 제주특별자치도 서귀포시_고령화비율및노령화지수현황
def fetchJejuPopulationInformation():
    # https://www.data.go.kr/iim/api/selectAPIAcountView.do 
    API_URL = "https://api.odcloud.kr/api/15051545/v1/uddi:70bc1765-8633-4021-ae84-e28ea48cc9f2"
    
    # Decoding
    API_KEY = "8OGTO7jd9Ih3JPZej0XBI0ITT2g6v1f1zxq7TibztJxeUYTyVjqSYAqePFeUP1eX7D5r2EKdlWU3w6Wfu9fKug=="
    
    # Parameters 설정
    # page=1&perPage=10 
    _d=dict()
    _d['page']= '1'
    _d['perPage']= '1'
    _d['serviceKey']= API_KEY
    _d['returnType']= 'JSON'
    
    # url encoding
    params=urllib.parse.urlencode(_d)
    
    # data fetching
    res=requests.get(API_URL + '?' + params)
    
    return res.json()

In [35]:
# %%writefile data/경기도_의정부시_인구현황_20200904.txt

import json
with open(os.path.join("data", "경기도_의정부시_인구현황_20200904.txt"), 'w') as f:
    json.dump(fetchUijeongbuPopulationInformation(), f)

In [36]:
import json
with open(os.path.join("data", "제주특별자치도_서귀포시_고령화비율및노령화지수현황_20200623.txt"), 'w') as f:
    json.dump(fetchJejuPopulationInformation(), f)

In [37]:
import pyspark

myConf=pyspark.SparkConf()
spark = pyspark.sql.SparkSession\
    .builder\
    .master("local")\
    .appName("myApp")\
    .config(conf=myConf)\
    .getOrCreate()

In [38]:
spark.sparkContext.getConf().getAll()

[('spark.master', 'local'),
 ('spark.driver.port', '55060'),
 ('spark.app.name', 'myApp'),
 ('spark.rdd.compress', 'True'),
 ('spark.app.startTime', '1632305531773'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.sql.warehouse.dir',
  'file:/Users/kangjuyoung/Desktop/workspace_jooyoung/university/labs3-2/bigdata-engineering/spark-warehouse'),
 ('spark.submit.pyFiles', ''),
 ('spark.executor.id', 'driver'),
 ('spark.submit.deployMode', 'client'),
 ('spark.driver.host', '192.168.0.5'),
 ('spark.ui.showConsoleProgress', 'true'),
 ('spark.app.id', 'local-1632305532584')]

In [41]:
import os 
myRdd=spark.sparkContext\
    .textFile(os.path.join("data","경기도_의정부시_인구현황_20200904.txt"))

myRdd.take(3)

['{"currentCount": 1, "data": [{"\\uad00\\ub9ac\\uae30\\uad00\\uba85": "\\uc758\\uc815\\ubd80\\uc2dc", "\\uad00\\ub9ac\\ubd80\\uc11c\\uba85": "\\uc2dc\\ubbfc\\ubd09\\uc0ac\\uacfc", "\\uad6c\\uc131\\ube44(\\uacc4)": "6.83", "\\uad6c\\uc131\\ube44(\\ub0a8)": "3.49", "\\uad6c\\uc131\\ube44(\\uc5ec)": "3.34", "\\ub370\\uc774\\ud130\\uae30\\uc900\\uc77c\\uc790": "2021-01-04", "\\ubd80\\uc11c\\uc804\\ud654\\ubc88\\ud638": "031-828-2466", "\\uc131\\ube44": "104.6", "\\uc138\\ub300\\ub2f9\\uc778\\uad6c": "1.63", "\\uc138\\ub300\\uc218": "19,412", "\\uc778\\uad6c\\uc218(\\uacc4)": "31,550", "\\uc778\\uad6c\\uc218(\\ub0a8)": "16,130", "\\uc778\\uad6c\\uc218(\\uc5ec)": "15,420", "\\ud589\\uc815\\uae30\\uad00": "\\uc758\\uc815\\ubd801\\ub3d9       "}], "matchCount": 14, "page": 1, "perPage": 1, "totalCount": 14}']